# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846914510534                   -0.70    4.5
  2   -7.852332712084       -2.27       -1.53    1.0
  3   -7.852645957776       -3.50       -2.53    2.8
  4   -7.852646677648       -6.14       -3.36    2.8
  5   -7.852646685993       -8.08       -4.78    1.2
  6   -7.852646686725       -9.14       -5.24    4.2
  7   -7.852646686730      -11.32       -5.94    2.0
  8   -7.852646686730      -12.54       -7.38    1.8
  9   -7.852646686730      -14.57       -7.79    2.2
 10   -7.852646686730   +  -15.05       -8.55    2.0
 11   -7.852646686730   +    -Inf       -9.29    2.8
 12   -7.852646686730   +  -15.05       -9.96    1.0
 13   -7.852646686730      -14.57       -9.23    1.0
 14   -7.852646686730   +  -14.75       -9.13    1.0
 15   -7.852646686730   +  -15.05       -9.15    1.0
 16   -7.852646686730      -15.05       -9.31    1.0
 17   -7.852646686730   +    -Inf      -10.03 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846794387229                   -0.70           4.8
  2   -7.852526610434       -2.24       -1.63   0.80    2.2
  3   -7.852636997931       -3.96       -2.71   0.80    1.0
  4   -7.852646449274       -5.02       -3.29   0.80    2.0
  5   -7.852646677476       -6.64       -4.13   0.80    1.8
  6   -7.852646686562       -8.04       -4.77   0.80    1.8
  7   -7.852646686717       -9.81       -5.72   0.80    1.5
  8   -7.852646686730      -10.91       -6.76   0.80    2.5
  9   -7.852646686730      -12.65       -7.63   0.80    2.2
 10   -7.852646686730      -14.75       -9.08   0.80    2.0
 11   -7.852646686730   +    -Inf       -8.64   0.80    3.5
 12   -7.852646686730   +  -14.57       -9.49   0.80    1.0
 13   -7.852646686730      -15.05      -10.34   0.80    1.0
 14   -7.852646686730   +  -15.05      -11.10   0.80    2.0
 15   -7.852646686730      -14.35      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.415347e+01     3.330773e+00
 * time: 0.4491748809814453
     1     1.072013e+00     1.759465e+00
 * time: 0.6942028999328613
     2    -1.290555e+00     2.112202e+00
 * time: 0.7224760055541992
     3    -3.844877e+00     1.631682e+00
 * time: 0.760814905166626
     4    -5.135011e+00     1.396647e+00
 * time: 0.7993960380554199
     5    -6.820694e+00     7.825370e-01
 * time: 0.8373310565948486
     6    -7.035438e+00     8.473043e-01
 * time: 0.8635520935058594
     7    -7.501412e+00     4.988218e-01
 * time: 0.889599084854126
     8    -7.642839e+00     2.534134e-01
 * time: 0.916254997253418
     9    -7.723512e+00     1.337986e-01
 * time: 0.9438130855560303
    10    -7.772445e+00     1.107918e-01
 * time: 0.9690020084381104
    11    -7.792412e+00     8.449653e-02
 * time: 0.9949550628662109
    12    -7.813372e+00     8.832470e-02
 * time: 1.020956039428711
    13    -7.828241e+00     7.266812e-02
 * time: 1.04780602455139

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846847481131                   -0.70    4.5
  2   -7.852313277518       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686674                   -2.54
  2   -7.852646686730      -10.26       -5.95
  3   -7.852646686730   +    -Inf      -12.19


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.2643237912216476e-13
|ρ_newton - ρ_scfv| = 3.481154601319785e-12
|ρ_newton - ρ_dm|   = 4.2547778511442593e-10
